# Twitter Data Updates

This Python code will be used to update and append the data sets created in `TwitterData-Start.ipynb` since we can only access the Twitter data of the last 7 days.

In [51]:
# Import Dependencies
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import tweepy

from config import consumer_key
from config import consumer_secret
from config import access_token
from config import access_token_secret
from config import token

In [52]:
# API Keys
consumer_key = consumer_key
consumer_secret = consumer_secret
access_token = access_token
access_token_secret = access_token_secret
token = token

## Count of Gas & Oil Tweets

In [53]:
query = "gas oil -is:retweet"
client = tweepy.Client(bearer_token=token)
counts = client.get_recent_tweets_count(query=query, 
                                        granularity='day')

In [54]:
Lst = []
# Dict = {}
for i in counts.data:
    Dict = {}
    Dict['Day'] = str(i["start"][0:10])
    Dict['Tweet Count'] = i["tweet_count"]
    Lst.append(Dict)
    # print(f'Day: ' + str(i["start"][0:10]) + ', Tweet Count: ' + str(i["tweet_count"]))

In [55]:
df = pd.DataFrame(Lst)
df = pd.DataFrame.from_dict(Lst)
df = pd.DataFrame.from_records(Lst)
# df.to_csv('tweet_count.csv', index=False) 
update_df = df.iloc[-2:-1]

In [56]:
update_df

,Day,Tweet Count
6,2022-11-30,7249


In [58]:
update_df.to_csv('tweet_count.csv', mode='a', index=False, header=False)

## Adding Recent Tweets

In [59]:
start = '2022-11-30T00:00:00.000Z'
end = '2022-12-01T00:00:00.00Z'

tweets_list = []

tweets = tweepy.Paginator(client.search_recent_tweets, query=query,
                              tweet_fields=['context_annotations', 'created_at'],
                          start_time=start, 
                          end_time=end,
                          max_results=100).flatten(limit=10000)

for tweet in tweets:
    tweets_list.append(tweet.text)
    
len(tweets_list)

7239

In [60]:
tweet_df = pd.DataFrame(tweets_list)
tweet_df.columns = ['2022-11-30']
tweet_df = tweet_df.drop_duplicates()
tweet_df.to_csv('2022-11-30.csv', index=True) 
len(tweet_df)

7090

In [61]:
all_tweets = pd.read_csv("./tweets.csv")

In [62]:
all_tweets = all_tweets.join(tweet_df, how='outer')
# all_tweets = all_tweets.dropna()

In [63]:
all_tweets = all_tweets.drop(columns=['Unnamed: 0'])
all_tweets

,2022-11-24,2022-11-25,2022-11-26,2022-11-27,2022-11-28,2022-11-29,2022-11-30
0,@BellaWallerstei They have been paying oil and...,@TrainerGonzo @Belinda71555585 It's that easy ...,@HouseGOP They tried to stick the gas price hi...,@GilHough @KamalaHarris https://t.co/LBgWDBW9xs,"@ChrisMartzWX For well over 40 years, I have p...",@POTUS Except that gas prices were on the rise...,@FinancialReview simply nonsense. Don't let th...
1,@BladeoftheS 3 The major difference between UK...,@davidmweissman Clueless! Right he inherited e...,@AnnieForTruth Yep. I remember when an attenda...,@NikkiHaley I agree use our own oil and coal a...,@WhiteHouse His executive orders on the energy...,@robreiner You really need help. Why are you s...,"@GavinNewsom You could end the gas tax, but yo..."
2,¡Oil &amp; Gas Reporter está disponible! https...,@laurenboebert Magic money? You mean from the ...,"@RBReich -\n...and the NATO Supremacists said,...",Nothing is being offered near Utah’s oil and g...,@manuel_rdgn @SEGOB_mx @adan_augusto @calderon...,TCS is a proud sponsor of the Oil &amp; Gatsby...,@POTUS Gas prices are down only because you ar...
3,@binnahar85 Europe wants to boycott Qatar? Are...,@thetruthsucks12 @haliwell51 @gmbutts No maybe...,@Princemoye1 I thought its about those that ki...,@Scott19490 @isaiahrmartin Yeah because relian...,rtr: U.S. seeks to limit flaring and methane l...,Should California tax oil profits? Gas spike h...,https://t.co/HEMuIIrFin Oil “Quality News Link...
4,@TheoFleury14 This was always about $$ and the...,You are full of it. You shut down our clean cr...,The big story. Proof that Westminster has been...,@RepBoebert Lauren Boebert’s income is almost ...,¡Oil &amp; Gas Reporter está disponible! https...,This is a classic example of Washington corrup...,@SenatorLankford You do know that US is the la...
...,...,...,...,...,...,...,...
7234,NaN,NaN,NaN,NaN,NaN,NaN,Oil Refiners More Than Doubled Their CA Profit...
7235,NaN,NaN,NaN,NaN,NaN,NaN,Oil #and #Gas #Jobs in United States\n- Progra...
7236,NaN,NaN,NaN,NaN,NaN,NaN,@HouseGOP @SteveScalise @RepBethVanDuyne US oi...
7237,NaN,NaN,NaN,NaN,NaN,NaN,The Interior Department on Monday proposed rul...


In [64]:
all_tweets.to_csv('tweets.csv')